# Categorical features

In the field of machine learning, categorical features play a crucial role in determining the predictive ability of a model. Categorical features are features that can take a limited number of values, such as color, gender or location. While these features can provide useful insights into patterns and relationships within data, they also pose unique challenges for machine learning models.

One of these challenges is the need to transform categorical features before they can be used by most models. This transformation involves converting categorical values into numerical values that can be processed by machine learning algorithm.

Another challenge is dealing with infrequent categories, which can lead to biased models. If a categorical feature has a large number of categories, but some of them are rare or appear infrequently in the data, the model may not be able to learn accurately from these categories, resulting in biased predictions and inaccurate results.

Despite these difficulties, categorical features are still an essential component in many use cases. When properly encoded and handled, machine learning models can effectively learn from patterns and relationships in categorical data, leading to better predictions.

This document provides an overview of three of the most commonly used transformations: one-hot encoding, ordinal encoding, and target encoding. It explains how to apply them in the skforecast package using scikit-learn encoders, which provide a convenient and flexible way to preprocess data. Additionally, it demonstrates how to use the native implementation of four popular gradient boosting frameworks – **LightGBM**, **scikit-learn's HistogramGradientBoosting**, **XGBoost**, and **CatBoost** – to handle categorical features directly in the model.

For a comprehensive demonstration of the use of categorical features in time series forecasting, check out the article [Forecasting time series with gradient boosting: Skforecast, XGBoost, LightGBM y CatBoost](https://www.cienciadedatos.net/documentos/py39-forecasting-time-series-with-skforecast-xgboost-lightgbm-catboost.html).

<script src="https://kit.fontawesome.com/d20edc211b.js" crossorigin="anonymous"></script>

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4;">

<p class="title">
    <i class="fa-circle-exclamation fa" style="font-size: 18px; color:#00b8d4;"></i>
    <b> &nbsp Note</b>
</p>

All transformations described in this document can be applied to the entire dataset independently of the forecaster. However, it is crucial to ensure that the transformations are learned exclusively from the training data to prevent information leakage. Furthermore, the same transformation should be applied to the input data during predictions. To reduce the likelihood of errors and ensure consistent application of transformations, it is advisable to include the transformation within the forecaster object, so that it is handled internally.
</div>

## Libraries and data

The dataset utilized in this user guide consists of information on the user count of a bicycle rental service, in addition to weather variables and holiday data. Two of the variables in the dataset, namely "holiday" and "weather", are categorical.

In [33]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent.parent))
str(Path.cwd().parent.parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [34]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5

from lightgbm import LGBMRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect

In [35]:
# Downloading data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-'
       'learning-python/master/data/bike_sharing_dataset_clean.csv')
data = pd.read_csv(url)

# Preprocess data
# ==============================================================================
data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')
data = data.set_index('date_time')
data = data.asfreq('H')
data = data.sort_index()
data['holiday'] = data['holiday'].astype(int)
data = data[['holiday', 'weather', 'temp', 'hum', 'users']]
data[['holiday', 'weather']] = data[['holiday', 'weather']].astype(str)
print(data.dtypes)
data.head(3)

holiday     object
weather     object
temp       float64
hum        float64
users      float64
dtype: object


,holiday,weather,temp,hum,users
date_time,,,,,
2011-01-01 00:00:00,0,clear,9.84,81.0,16.0
2011-01-01 01:00:00,0,clear,9.02,80.0,40.0
2011-01-01 02:00:00,0,clear,9.02,80.0,32.0


To simplify the example, only a portion of the data is used.

In [36]:
# Split train-test
# ==============================================================================
start_train = '2012-06-01 00:00:00'
end_train = '2012-07-31 23:59:00'
end_test = '2012-08-15 23:59:00'
data_train = data.loc[start_train:end_train, :]
data_test  = data.loc[end_train:end_test, :]

print(f"Dates train : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates test  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Dates train : 2012-06-01 00:00:00 --- 2012-07-31 23:00:00  (n=1464)
Dates test  : 2012-08-01 00:00:00 --- 2012-08-15 23:00:00  (n=360)


## One Hot Encoding

One hot encoding, also known as dummy encoding or one-of-K encoding, consists in replacing the categorical variable by a group of binary variables which take value 0 or 1, to indicate if a certain category is present in an observation. For example, suppose a dataset that includes a categorical variable called "color" with possible values of "red," "blue," and "green." Using one hot encoding, this variable is converted into three binary variables such as "color_red," "color_blue," and "color_green," where each variable takes a value of 0 or 1 depending on the category.

The [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) class in scikit-learn can be used to transform each categorical feature with *n* possible values into *n* new binary features, with one of them taking the value 1, and all others 0. The `OneHotEncoder` can be configured to handle certain corner cases, including unknown categories, missing values, and infrequent categories.

+ When `handle_unknown='ignore'` and `drop` is not `None`, unknown categories are encoded as zeros. Additionally, if a feature contains both `np.nan` and `None`, they will be considered separate categories.

+ It supports aggregating infrequent categories into a single output for each feature. The parameters to enable the gathering of infrequent categories are `min_frequency` and `max_categories`. By setting `handle_unknown` to 'infrequent_if_exist', unknown categories will be considered infrequent.

+ To avoid collinearity between features, it is possible to drop one of the categories per feature using the `drop` argument. This is particularly important when using linear models.


[ColumnTransformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) in scikit-learn provide a powerful way to define transformations and apply them to specific features. By encapsulating the `OneHotEncoder` within a `ColumnTransformer` object, it can be passed to a forecaster using the `transformer_exog` argument.

In [37]:
# ColumnTransformer with one-hot encoding
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using one-hot encoding. Numerical features are left untouched. For binary
# features, only one column is created.
one_hot_encoder = make_column_transformer(
                    (
                        OneHotEncoder(sparse_output=False, drop='if_binary'),
                        make_column_selector(dtype_exclude=np.number)
                    ),
                    remainder="passthrough",
                    verbose_feature_names_out=False,
                ).set_output(transform="pandas")

In [38]:
# Create and fit forecaster with a transformer for exogenous features
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']

forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=123),
                lags = 5,
                transformer_exog = one_hot_encoder
             )

forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

forecaster

ForecasterAutoreg 
Regressor: LGBMRegressor(random_state=123) 
Lags: [1 2 3 4 5] 
Transformer for y: None 
Transformer for exog: ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='if_binary',
                                               sparse_output=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000011DEBBC2FE0>)],
                  verbose_feature_names_out=False) 
Window size: 5 
Weight function included: False 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['holiday', 'weather', 'temp', 'hum'] 
Training range: [Timestamp('2011-01-01 00:00:00'), Timestamp('2012-07-31 23:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: H 
Regressor parameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_ty

Once the forecaster has been trained, the transformer can be inspected (feature_names_in, feature_names_out, ...) by accessing the `transformer_exog` attribute.

In [39]:
# Access to the transformer used for exogenous features
# ==============================================================================
print(forecaster.transformer_exog.get_feature_names_out())
forecaster.transformer_exog

['holiday_1' 'weather_clear' 'weather_mist' 'weather_rain' 'temp' 'hum']


ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='if_binary',
                                               sparse_output=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000011DEBBC2FE0>)],
                  verbose_feature_names_out=False)

In [40]:
# Predictions
# ==============================================================================
forecaster.predict(steps=3, exog=data_test[exog_features])

2012-08-01 00:00:00    88.946940
2012-08-01 01:00:00    59.848451
2012-08-01 02:00:00    28.870817
Freq: H, Name: pred, dtype: float64

<script src="https://kit.fontawesome.com/d20edc211b.js" crossorigin="anonymous"></script>

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4;">

<p class="title">
    <i class="fa-circle-exclamation fa" style="font-size: 18px; color:#00b8d4;"></i>
    <b> &nbsp Note</b>
</p>

Applying a transformation to the entire dataset independent of the forecaster is feasible. However, it is crucial to ensure that the transformations are learned exclusively from the training data to prevent information leakage. Additionally, the same transformation should be applied to the input data during predictions. Therefore, it is advisable to include the transformation within the forecaster, so that it is handled internally. This approach ensures consistency in the application of transformations and reduces the likelihood of errors.

To examine how data is being transformed, it is possible to use the method <code>create_train_X_y</code> to generate the matrices the forecaster is using to train the model. This approach enables gaining insight into the specific data manipulations that occur during the training process.

</div>

In [41]:
# Create training matrices
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(
                        y = data.loc[:end_train, 'users'],
                        exog = data.loc[:end_train, exog_features]
                   )
print(X_train.dtypes)
X_train.head()

lag_1            float64
lag_2            float64
lag_3            float64
lag_4            float64
lag_5            float64
holiday_1        float64
weather_clear    float64
weather_mist     float64
weather_rain     float64
temp             float64
hum              float64
dtype: object


,lag_1,lag_2,lag_3,lag_4,lag_5,holiday_1,weather_clear,weather_mist,weather_rain,temp,hum
date_time,,,,,,,,,,,
2011-01-01 05:00:00,1.0,13.0,32.0,40.0,16.0,0.0,0.0,1.0,0.0,9.84,75.0
2011-01-01 06:00:00,1.0,1.0,13.0,32.0,40.0,0.0,1.0,0.0,0.0,9.02,80.0
2011-01-01 07:00:00,2.0,1.0,1.0,13.0,32.0,0.0,1.0,0.0,0.0,8.20,86.0
2011-01-01 08:00:00,3.0,2.0,1.0,1.0,13.0,0.0,1.0,0.0,0.0,9.84,75.0
2011-01-01 09:00:00,8.0,3.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,13.12,76.0


In [42]:
# Transform exogenous features using the transformer outside the forecaster
# ==============================================================================
exog_transformed = one_hot_encoder.fit_transform(data.loc[:end_train, exog_features])
exog_transformed.head()

,holiday_1,weather_clear,weather_mist,weather_rain,temp,hum
date_time,,,,,,
2011-01-01 00:00:00,0.0,1.0,0.0,0.0,9.84,81.0
2011-01-01 01:00:00,0.0,1.0,0.0,0.0,9.02,80.0
2011-01-01 02:00:00,0.0,1.0,0.0,0.0,9.02,80.0
2011-01-01 03:00:00,0.0,1.0,0.0,0.0,9.84,75.0
2011-01-01 04:00:00,0.0,1.0,0.0,0.0,9.84,75.0


## Ordinal encoding

Ordinal encoding is a technique used to convert categorical variables into numerical variables. Each category is assigned a unique numerical value based on its order or rank, as determined by a chosen criterion such as frequency or importance. This encoding method is particularly useful when categories have a natural order or ranking, such as with educational levels. However, it is important to note that the numerical values assigned to each category do not represent any inherent numerical difference between them, but simply provide a numerical representation.

The Scikit-learn library provides the [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) class, which enables users to replace categorical variables with ordinal numbers, ranging from 0 to n_categories-1. Additionally, this class includes the `encoded_missing_value` parameter that allows encoding of missing values. It is important to note that this implementation assigns numbers to categories arbitrarily, based on a first-seen, first-served basis. Therefore, users should exercise caution when interpreting the numerical values assigned to the categories. Other implementations such as the one in [Feature-engine](https://feature-engine.trainindata.com/en/1.0.x/encoding/OrdinalEncoder.html) numbers can be ordered based on the mean of the target.

In [43]:
# ColumnTransformer with ordinal encoding
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using ordinal encoding. Numerical features are left untouched. Missing values
# are encoded as -1. If a new category is found in the test set, it is encoded
# as -1.
ordinal_encoder = make_column_transformer(
                    (
                        OrdinalEncoder(
                            handle_unknown='use_encoded_value',
                            unknown_value=-1,
                            encoded_missing_value=-1
                        ),
                        make_column_selector(dtype_exclude=np.number)
                    ),
                    remainder="passthrough",
                    verbose_feature_names_out=False,
                ).set_output(transform="pandas")

In [44]:
# Create and fit forecaster with a transformer for exogenous features
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']

forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=123),
                lags = 5,
                transformer_exog = ordinal_encoder
             )

forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

forecaster

ForecasterAutoreg 
Regressor: LGBMRegressor(random_state=123) 
Lags: [1 2 3 4 5] 
Transformer for y: None 
Transformer for exog: ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinalencoder',
                                 OrdinalEncoder(encoded_missing_value=-1,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000011DEBBC3CD0>)],
                  verbose_feature_names_out=False) 
Window size: 5 
Weight function included: False 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['holiday', 'weather', 'temp', 'hum'] 
Training range: [Timestamp('2011-01-01 00:00:00'), Timestamp('2012-07-31 23:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: H 
Regressor paramet

In [45]:
# Create training matrices
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(
                        y = data.loc[:end_train, 'users'],
                        exog = data.loc[:end_train, exog_features]
                   )
print(X_train.dtypes)
X_train.head()

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
holiday    float64
weather    float64
temp       float64
hum        float64
dtype: object


,lag_1,lag_2,lag_3,lag_4,lag_5,holiday,weather,temp,hum
date_time,,,,,,,,,
2011-01-01 05:00:00,1.0,13.0,32.0,40.0,16.0,0.0,1.0,9.84,75.0
2011-01-01 06:00:00,1.0,1.0,13.0,32.0,40.0,0.0,0.0,9.02,80.0
2011-01-01 07:00:00,2.0,1.0,1.0,13.0,32.0,0.0,0.0,8.20,86.0
2011-01-01 08:00:00,3.0,2.0,1.0,1.0,13.0,0.0,0.0,9.84,75.0
2011-01-01 09:00:00,8.0,3.0,2.0,1.0,1.0,0.0,0.0,13.12,76.0


Once the forecaster has been trained, the transformer can be inspected by accessing the `transformer_exog` attribute.

In [46]:
# Predictions
# ==============================================================================
forecaster.predict(steps=3, exog=data_test[exog_features])

2012-08-01 00:00:00    89.096098
2012-08-01 01:00:00    57.749964
2012-08-01 02:00:00    29.263922
Freq: H, Name: pred, dtype: float64

## Target encoding

Target encoding is a technic that encodes categorical variables based on the relationship between the categories and the target variable. Each category is encoded based on a shrinked estimate of the average target values for observations belonging to the category. The encoding scheme mixes the global target mean with the target mean conditioned on the value of the category.

For example, suppose a categorical variable "City" with categories "New York," "Los Angeles," and "Chicago," and a target variable "Salary." One can calculate the mean salary for each city category based on the training data, and use these mean values to encode the categories.

This encoding scheme is useful with categorical features with high cardinality, where one-hot encoding would inflate the feature space making it more expensive for a downstream model to process. A classical example of high cardinality categories are location based such as zip code or region.

The [TargetEncoder](https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.TargetEncoder.html) class is available in Scikit-learn (since version 1.3). `TargetEncoder` considers missing values, such as `np.nan` or `None`, as another category and encodes them like any other category. Categories that are not seen during fit are encoded with the target mean, i.e. `target_mean_`. A more detailed description of target encoding can be found in the [scikit-learn user guide](https://scikit-learn.org/dev/modules/preprocessing.html#target-encoder).

<script src="https://kit.fontawesome.com/d20edc211b.js" crossorigin="anonymous"></script>

<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100;">

<p class="title">
    <i class="fa-triangle-exclamation fa" style="font-size: 18px; color:#ff9100;"></i>
    <b> &nbsp Warning</b>
</p>

We are currently working to allow this type of transformation within skforecast.

</div>

## Native implementation for categorical features

Some machine learning models, including **XGBoost**, **LightGBM**, **CatBoost**, and **HistGradientBoostingRegressor**, offer built-in methods to handle categorical features, however, they assume that the input categories are integers starting from 0 up to the number of categories [0, 1, ..., n_categories-1]. In practice, categorical variables are not encoded with numbers but with strings, so an intermediate transformation step is necessary. Two options are:

+ Set columns with categorical variables to `category` type. Internally, this data structure consists of an array of categories and an array of integer values (codes) that point to the actual value of the array of categories. That is, internally they are a numeric array with a mapping that relates each value to a category. Models are able to automatically identify the columns of type `category` and access their internal codes. This approach is applicable with **XGBoost**, **LightGBM** and **CatBoost**.

+ Preprocess the categorical columns with an `OrdinalEncoder` to transform their values into integers and explicitly indicate that the columns should be treated as categorical. Skforecast allows it by using the `fit_kwargs` argument.

<script src="https://kit.fontawesome.com/d20edc211b.js" crossorigin="anonymous"></script>

<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100;">

<p class="title">
    <i class="fa-triangle-exclamation fa" style="font-size: 18px; color:#ff9100;"></i>
    <b> &nbsp Warning</b>
</p>

When deploying models in production, it is strongly advised to avoid using automatic detection based on pandas `category` type columns. While pandas does provide internal coding for these columns, it is not consistent across different data sets and may vary depending on the categories present in each data set. Therefore, it is crucial to be mindful of this issue and to implement appropriate measures to ensure consistency in the coding of categorical features when deploying models in production.

As of the writing of this documentation, the authors have only observed that LightGBM internally manages changes in the coding of categories. More details on this issue can be found in  <a href=https://github.com/microsoft/LightGBM/issues/2826)>github issue</a> and  <a href=https://stackoverflow.com/questions/60191643/are-lightgbm-treating-pandas-categoricals-based-on-name-or-cat-code-value>stackoverflow</a>.

If the user still wishes to rely on automatic detection of categorical features based on pandas data types, categorical variables must be encoded first as integers (ordinal encoding) and then stored as category type. This is needed because skforecast makes internal use of numerical numpy array to speed the computation.

</div>

### LightGBM

**Encoding categories as integers and explicity indicating the name of the categorical features (recommended)**

When creating a forecaster using `LGBMRegressor`, it is necessary to specify the names of the categorical columns using the `fit_kwargs` argument. This is because the `categorical_feature` argument is only specified in the `fit` method of `LGBMRegressor`, and not during its initialization.

In [47]:
# Transformer: ordinal encoding
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using ordinal encoding. Numerical features are left untouched. Missing values
# are encoded as -1. If a new category is found in the test set, it is encoded
# as -1.
categorical_features = data.select_dtypes(exclude=[np.number]).columns.tolist()
transformer_exog = make_column_transformer(
                        (
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            categorical_features
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

In [48]:
# Create and fit forecaster indicating the categorical features
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']

forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=963),
                lags = 5,
                transformer_exog = transformer_exog,
                fit_kwargs={'categorical_feature': categorical_features}
             )
            
forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

c:\anaconda\envs\skforecast_p10\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\anaconda\envs\skforecast_p10\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\anaconda\envs\skforecast_p10\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


In [49]:
# Predictions
# ==============================================================================
forecaster.predict(steps=3, exog=data_test[exog_features])

2012-08-01 00:00:00    89.451199
2012-08-01 01:00:00    49.589097
2012-08-01 02:00:00    22.181453
Name: pred, dtype: float64

**Allow the model to automatically detect categorical features (not recommended)**

<script src="https://kit.fontawesome.com/d20edc211b.js" crossorigin="anonymous"></script>

<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100;">

<p class="title">
    <i class="fa-triangle-exclamation fa" style="font-size: 18px; color:#ff9100;"></i>
    <b> &nbsp Warning</b>
</p>

Handle categorical variables by relying on the automatic detection of `category` data types can be achieved by setting `categorical_features='auto'` during model initialization. However, this approach may lead to significant issues if the model is exposed to new datasets that have different pandas encoding for categorical columns than the one used during training. Therefore, it's crucial to ensure that the encoding is consistent between the training and the testing datasets to avoid any potential errors.

</div>

In [18]:
# Transformer: ordinal encoding and cast to category type
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using ordinal encoding. Numerical features are left untouched. Missing values
# are encoded as -1. If a new category is found in the test set, it is encoded
# as -1. After the encoding, the features are cast again to category type so they
# can beidentified as categorical features by the regressor.

pipeline_categorical = make_pipeline(
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            FunctionTransformer(
                                func=lambda x: x.astype('category'),
                                feature_names_out= 'one-to-one'
                            )
                       )

transformer_exog = make_column_transformer(
                        (
                            pipeline_categorical,
                            make_column_selector(dtype_exclude=np.number)
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

In [19]:
# Create and fit forecaster
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']

forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(categorical_features='auto', random_state=963),
                lags = 5,
                transformer_exog = transformer_exog
             )
            
forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

c:\anaconda\envs\skforecast_p10\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_features in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [20]:
# Predictions
# ==============================================================================
forecaster.predict(steps=3, exog=data_test[exog_features])

2012-08-01 00:00:00    88.946940
2012-08-01 01:00:00    59.848451
2012-08-01 02:00:00    28.870817
Freq: H, Name: pred, dtype: float64

As with any other forecaster, the matrices used during model training can created with `create_train_X_y`.

In [21]:
# Create training matrices
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(
                        y = data.loc[:end_train, 'users'],
                        exog = data.loc[:end_train, exog_features]
                   )
X_train.head()

,lag_1,lag_2,lag_3,lag_4,lag_5,holiday,weather,temp,hum
date_time,,,,,,,,,
2011-01-01 05:00:00,1.0,13.0,32.0,40.0,16.0,0,1,9.84,75.0
2011-01-01 06:00:00,1.0,1.0,13.0,32.0,40.0,0,0,9.02,80.0
2011-01-01 07:00:00,2.0,1.0,1.0,13.0,32.0,0,0,8.20,86.0
2011-01-01 08:00:00,3.0,2.0,1.0,1.0,13.0,0,0,9.84,75.0
2011-01-01 09:00:00,8.0,3.0,2.0,1.0,1.0,0,0,13.12,76.0


### Scikit-learn HistogramGradientBoosting

When creating a forecaster using `HistogramGradientBoosting`, the names of categorical columns should be specified during the instantiation of the regressor by passing them as a list to the `categorical_feature` argument.

In [22]:
# Transformer: ordinal encoding
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using ordinal encoding. Numerical features are left untouched. Missing values
# are encoded as -1. If a new category is found in the test set, it is encoded
# as -1.
categorical_features = data.select_dtypes(exclude=[np.number]).columns.tolist()
transformer_exog = make_column_transformer(
                        (
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            categorical_features
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

In [23]:
# Create and fit forecaster indicating the categorical features
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']

forecaster = ForecasterAutoreg(
                regressor = HistGradientBoostingRegressor(
                                categorical_features=categorical_features,
                                random_state=963
                            ),
                lags = 5,
                transformer_exog = transformer_exog
             )
            
forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

`HistGradientBoostingRegressor` stores boolean mask that indicates what features have been considered as categorical. `None` if there are no categorical features.

In [24]:
forecaster.regressor.is_categorical_

array([False, False, False, False, False,  True,  True, False, False])

In [25]:
# Predictions
# ==============================================================================
forecaster.predict(steps=3, exog=data_test[exog_features])

2012-08-01 00:00:00    99.185547
2012-08-01 01:00:00    71.914255
2012-08-01 02:00:00    43.342723
Freq: H, Name: pred, dtype: float64

### XGBoost

**Encoding categories as integers and explicity indicating the categorical features (recommended)**

As of the time of writing, the `XGBRegressor` module does not offer an explicit option for specifying the names of categorical features. Instead, the feature types are indicated by passing a list of strings to the `feature_types` argument, where 'c' denotes categorical and 'q' denotes numeric features. Moreover, the `enable_categorical` argument must be set to True.

Determining the position of each feature to create the list of feature types can be challenging, as the shape of the data matrix depends on the number of lags used and the transformations applied to the exogenous variables. A workaround involves creating a dummy forecaster and using the `create_train_X_y` method with a small sample of data to determine the final position of each feature. This approach can help to accurately specify the feature types.

In [ ]:
# Transformer: ordinal encoding
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using ordinal encoding. Numerical features are left untouched. Missing values
# are encoded as -1. If a new category is found in the test set, it is encoded
# as -1.
categorical_features = data.select_dtypes(exclude=[np.number]).columns.tolist()
transformer_exog = make_column_transformer(
                        (
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            categorical_features
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

A first forecaster is created without indicating the argument `feature_types`.

In [61]:
# Create forecaster
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']
forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(
                              tree_method='hist',
                              random_state=12345,
                              enable_categorical=True,
                            ),
                lags = 5,
                transformer_exog = transformer_exog
             )
            
forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

In [62]:
# Create training matrices with a sample of the data
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(
                        y = data.loc[:end_train, 'users'][:10],
                        exog = data.loc[:end_train, exog_features][:10]
                   )
X_train.head(3)

,lag_1,lag_2,lag_3,lag_4,lag_5,holiday,weather,temp,hum
date_time,,,,,,,,,
2011-01-01 05:00:00,1.0,13.0,32.0,40.0,16.0,0,1,9.84,75.0
2011-01-01 06:00:00,1.0,1.0,13.0,32.0,40.0,0,0,9.02,80.0
2011-01-01 07:00:00,2.0,1.0,1.0,13.0,32.0,0,0,8.20,86.0


Create a list to identify what columns in the training matrix is numeric and categoric.

In [63]:
feature_types = ['c' if col in categorical_features else 'q' for col in X_train.columns]
feature_types

['q', 'q', 'q', 'q', 'q', 'c', 'c', 'q', 'q']

In [66]:
Update the regressor parameters using the forecaster's method `set_params`.

SyntaxError: unterminated string literal (detected at line 1) (2387882118.py, line 1)

In [65]:
forecaster.set_params({'feature_types': feature_types})
forecaster

ForecasterAutoreg 
Regressor: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None,
             feature_types=['q', 'q', 'q', 'q', 'q', 'c', 'c', 'q', 'q'],
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=55, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=12345, ...) 
Lags: [1 2 3 4 5] 
Transformer for y: None 
Transformer for exog: ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinalencoder',
               

**Allow the model to automatically detect categorical features (not recommended)**

<script src="https://kit.fontawesome.com/d20edc211b.js" crossorigin="anonymous"></script>

<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100;">

<p class="title">
    <i class="fa-triangle-exclamation fa" style="font-size: 18px; color:#ff9100;"></i>
    <b> &nbsp Warning</b>
</p>

Handle categorical variables by relying on the automatic detection of `category` data types can be achieved by setting `enable_categorical=True` during model initialization. However, this approach may lead to significant issues if the model is exposed to new datasets that have different pandas encoding for categorical columns than the one used during training. Therefore, it's crucial to ensure that the encoding is consistent between the training and the testing datasets to avoid any potential errors.

</div>

In [26]:
# Transformer: ordinal encoding and cast to category type
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using ordinal encoding. Numerical features are left untouched. Missing values
# are encoded as -1. If a new category is found in the test set, it is encoded
# as -1. After the encoding, the features are cast again to category type so they
# can beidentified as categorical features by the regressor.

pipeline_categorical = make_pipeline(
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            FunctionTransformer(
                                func=lambda x: x.astype('category'),
                                feature_names_out= 'one-to-one'
                            )
                       )

transformer_exog = make_column_transformer(
                        (
                            pipeline_categorical,
                            make_column_selector(dtype_exclude=np.number)
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

In [27]:
# Create and fit forecaster
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']

forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(
                                enable_categorical=True,
                                tree_method='hist',
                                random_state=963
                            ),
                lags = 5,
                transformer_exog = transformer_exog
             )
            
forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

In [28]:
# Predictions
# ==============================================================================
forecaster.predict(steps=3, exog=data_test[exog_features])

2012-08-01 00:00:00    77.470787
2012-08-01 01:00:00    40.735706
2012-08-01 02:00:00    13.448755
Freq: H, Name: pred, dtype: float64

## CatBoost

Unfortunately, the current versions of `ForecasterAutoreg`, `ForecasterAutoregCustom`, and the multiseries equivalent are not compatible with CatBoost's native handling of categorical features. The issue arises because CatBoost only accepts categorical features as integers, while skforecast converts input data to floats for faster computation using numpy arrays in the internal recursive prediction process. As a result, there is a mismatch in data types between the two frameworks, leading to incompatibility.

In [29]:
# Transformer: ordinal encoding
# ==============================================================================
# A ColumnTransformer is used to transform categorical features (no numerical)
# using ordinal encoding. Numerical features are left untouched. Missing values
# are encoded as -1. If a new category is found in the test set, it is encoded
# as -1.
categorical_features = data.select_dtypes(exclude=[np.number]).columns.tolist()
transformer_exog = make_column_transformer(
                        (
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            categorical_features
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                   ).set_output(transform="pandas")

In [31]:
# Create and fit forecaster
# ==============================================================================
exog_features = ['holiday', 'weather', 'temp', 'hum']

forecaster = ForecasterAutoregDirect(
                regressor = CatBoostRegressor(
                                cat_features=categorical_features,
                                n_estimators=5,
                                random_state=963,
                                silent=True,
                                allow_writing_files=False
                            ),
                lags = 5,
                steps = 3,
                transformer_exog = transformer_exog
             )
            
forecaster.fit(
    y = data.loc[:end_train, 'users'],
    exog = data.loc[:end_train, exog_features]
)

In [32]:
# Predictions
# ==============================================================================
forecaster.predict(steps=3, exog=data_test[exog_features])

CatBoostError: 'data' is numpy array of floating point numerical type, it means no categorical features, but 'cat_features' parameter specifies nonzero number of categorical features

In [ ]:
%%html
<style>
.jupyter-wrapper .jp-CodeCell .jp-Cell-inputWrapper .jp-InputPrompt {display: none;}
</style>